In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


<img src='../../../common/logo_DH.png' align='left' width=35%/>

# Series de Tiempo - Práctica independiente


#### En esta práctica independiente vas a poner en práctica tu conocimiento de series de tiempo. 

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt

from scipy import stats
from statistics import mode

from sklearn.model_selection import train_test_split

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA

import warnings
warnings.filterwarnings('ignore')


En este caso, nuestra variable de interés en la cantidad de pasajeros de una compañía aérea.

Aplicá lo que aprendiste en la clase de series de tiempo y desarrolla un modelo para esta serie de tiempo.

In [ ]:
df = pd.read_csv('../Data/AirPassengers.csv')
df.shape

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.rename(columns={'#Passengers':'Passengers', 'Month':'Date'}, inplace=True)

df.head()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

df['Date'].dtype

In [ ]:
df = df.sort_values(by = "Date")

In [ ]:
df['Month'] = pd.DatetimeIndex(df['Date']).month
df.head()

In [ ]:
df.index = pd.PeriodIndex(df.Date, freq='M')
df.head()

Vamos a definir una función que plotea series de tiempo:

In [ ]:
# Función que plotea la serie:
def plot_df(df, x, y, title="", xlabel='Fecha', ylabel='Valor', dpi=100):
    plt.figure(figsize=(16,5), dpi=dpi)
    plt.plot(x, y, color='tab:red')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.show()

In [ ]:
plot_df(df, x=df.Date, y=df.Passengers, title="Passengers")

In [ ]:
df["timeIndex"] = pd.Series(np.arange(len(df['Passengers'])), index=df.index)
df.head()

In [ ]:
df.tail()

In [ ]:
dummies_mes = pd.get_dummies(df['Month'], drop_first=True, prefix='Month')
df = df.join(dummies_mes)
df.sample(10)

In [ ]:
df_train, df_test = train_test_split(df, test_size=12, random_state=42, shuffle=False)

In [ ]:
df_train.tail()

In [ ]:
df_test.head()

Creá las transformaciones logarítmicas de priceMod tanto para el set de entrenamiento como para el set de testeo.

In [ ]:
df_train['log_Passengers'] = np.log(df_train['Passengers'])
df_test['log_Passengers'] = np.log(df_test['Passengers'])

In [ ]:
plot_df(df_train, x=df_train.Date, y=df_train['log_Passengers'],\
    title='Log de Passengers del train set')

Vemos que la dispersión de la serie se estabilizó significativamente en t. 

Ahora entrená un modelo lineal entre la serie transformada y la dummy de tiempo y analizá el summary.

In [ ]:
model_log = smf.ols('log_Passengers ~ timeIndex',\
                          data = df_train).fit()

In [ ]:
model_log.summary()

Agregá las predicciones del modelo en el set de entrenamiento y de testeo con y sin back-transformation:

In [ ]:
df_train['model_log'] = model_log.predict(df_train[["timeIndex"]])
df_test['model_log'] = model_log.predict(df_test[["timeIndex"]])

In [ ]:
df_train['back_model_log'] = np.exp(df_train['model_log'])
df_test['back_model_log'] = np.exp(df_test['model_log'])

Ploteá las predicciones vs. las series reales, tanto en el set de entrenamiento como en el de testeo.

In [ ]:
df_train.plot(kind = "line", x = "Date", y = ['log_Passengers', 'model_log']);

In [ ]:
df_train.plot(kind = "line", x = "Date", y = ['Passengers', 'back_model_log']);

In [ ]:
df_test.plot(kind = "line", x = "Date", y = ['log_Passengers', 'model_log']);

In [ ]:
df_test.plot(kind = "line", x = "Date", y = ['Passengers', 'back_model_log']);

Creamos la función para calcular el RMSE:

In [ ]:
def RMSE(predicted, actual):
    mse = (predicted - actual) ** 2
    rmse = np.sqrt(mse.sum() / mse.count())
    return rmse

Guardá el resultado en un DataFrame:

In [ ]:
# POR FAVOR COMPLETÁ CON TU CÓDIGO:
df_Results = pd.DataFrame(columns = ["Model", "RMSE"])
df_Results.loc[0, "Model"] = "Log"
df_Results.loc[0, "RMSE"] = RMSE(df_test.back_model_log, df_test.Passengers)
df_Results

Ahora entrená un modelo agregando variables de estacionalidad mensual y agregá el RMSE en el DataFrame de resultados. 

In [ ]:
model_log_est = smf.ols('log_Passengers ~ timeIndex + Month_3 + Month_4 + Month_5 + Month_6 + Month_7 + Month_8 + Month_9 + Month_11',\
                          data = df_train).fit()


In [ ]:
model_log_est.summary()

Comentario: recordá que podés usar el método predict del modelo para realizar predicciones.
Al método le tenés que pasar el DataFrame y especificar las columnas a incluir. 

Hacé las predicciones en el set de entrenamiento y testeo y almacená los resultados en ambos DataFrames:

In [ ]:
df_train['model_log_est'] = model_log_est.predict(df_train[["timeIndex",\
                                              "Month_3", "Month_4", "Month_5", "Month_6",\
                                               "Month_7","Month_8", "Month_9", "Month_11"]])


df_test['model_log_est'] = model_log_est.predict(df_test[["timeIndex",\
                                              "Month_3", "Month_4", "Month_5", "Month_6",\
                                               "Month_7","Month_8", "Month_9", "Month_11"]])

Comentario: recordá que para hacer back transformation de una transformación logarítmica tenés que usar la función exponencial. 

Almacená en tus DataFrames los modelos con back transformation. 

In [ ]:
df_train['back_model_log_est'] = np.exp(df_train['model_log_est'])
df_test['back_model_log_est'] = np.exp(df_test['model_log_est'])

Plotea el modelo con y sin back transformation para el set de entrenamiento:

In [ ]:
df_train.plot(kind = "line", x = "Date", y = ['log_Passengers', 'model_log_est']);

In [ ]:
df_train.plot(kind = "line", x = "Date", y = ['Passengers', 'back_model_log_est']);

Plotea el modelo con y sin back transformation para el set de testeo:

In [ ]:
df_test.plot(kind = "line", x = "Date", y = ['log_Passengers', 'model_log_est']);

In [ ]:
df_test.plot(kind = "line", x = "Date", y = ['Passengers', 'back_model_log_est']);

Calculá el RMSE del modelo con transformación logarítmica y estacionalidad mensual y agregala al DataFrame de resultados: 

In [ ]:
df_Results.loc[1, "Model"] = "Log + Est"
df_Results.loc[1, "RMSE"] = RMSE(df_test.back_model_log_est, df_test.Passengers)
df_Results